In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import PIL
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import models
from SampleNN import *
%matplotlib inline
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
import torchvision
from torchvision import transforms
from tqdm import tqdm_notebook

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

testset = torchvision.datasets.CIFAR10(root='/home/yanzy/SEU_NNFailure/CIFAR_10/data', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,
                                         shuffle=False, num_workers=2, pin_memory=False)

classes = ('plane', 'car', 'bird', 'cat',
        'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [9]:
sample_net = models.nin(N=16, m=8).to(device)
State_File = torch.load('good.pt')
lol = sample_net.state_dict()
a = lol.keys()
#print (len(a))
gt = State_File
b = gt.keys()
for i in range(len(a)):
    lol[a[i]] = gt[b[i]]
sample_net.load_state_dict(lol)

In [14]:
def val(N=8, m=8):
    torch.cuda.empty_cache()
    TMP = protectStateDict(sample_net)
    sampleStateDict(sample_net,N=N,m=m)
    sample_net.eval()
    correct = 0
    total = 0
    count = 0
    with torch.no_grad():
        for data in tqdm_notebook(testloader):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            images = mSample(N=N,m=m)(images)
            outputs = sample_net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            count += labels.size(0)
            correct += (predicted == labels).sum().item()
            #print correct, total
    sample_net.load_state_dict(TMP)
    torch.cuda.empty_cache()

    print('Accuracy of the network on the %d test images: %.3f %%' % (total,
        100.0 * correct / total))
    return 100.0 * correct / total

In [15]:
val(N=16,m=8)


Accuracy of the network on the 10000 test images: 88.290 %


88.29

In [12]:
float_net = models.nin_float().to(device)
float_net.load_state_dict(gt)
def val_float():
    float_net.eval()
    correct = 0
    total = 0
    count = 0
    with torch.no_grad():
        for data in tqdm_notebook(testloader):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = float_net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            count += labels.size(0)
            correct += (predicted == labels).sum().item()
            #print correct, total
            if count >= 10000:
                print 'alive'
                count = 0

    print('Accuracy of the network on the %d test images: %.3f %%' % (total,
        100.0 * correct / total))
    return 100.0 * correct / total

In [13]:
val_float()

alive

Accuracy of the network on the 10000 test images: 88.510 %


88.51